# Custom Battery Cell Metadata

In this notebook, we define two custom battery cells (R2032 coin cells) using JSON-LD, enriched with semantic information from ontologies. We then parse this data into an RDF graph and query it using SPARQL.

This example illustrates:
- How to describe battery cells using ontology terms
- How JSON-LD is interpreted as semantic triples
- How to use SPARQL to explore structured linked data
- How to extend this structure for data integration or digital twins


## Describe Battery Cells Using JSON-LD

Below are two JSON-LD descriptions of battery cells:

- An LFP/Graphite R2032 coin cell
- An LNO/Graphite R2032 coin cell

These objects use ontology-defined terms such as:
- `BatteryCell`, `Electrode`, `Graphite`, `LithiumIronPhosphate`
- `NominalVoltage`, `hasProperty`, and `emmo:Volt`

We also use `schema:name` and `schema:manufacturer` for basic metadata.

Each block is fully self-contained and semantically interpretable.

In [30]:
jsonld_LFPGr = {
            "@context": "https://w3id.org/emmo/domain/battery/context",
            "@type": "BatteryCell",
            "schema:name": "My LFP-Graphite R2032 Coin Cell",
            "schema:manufacturer": {
               "@id": "https://www.wikidata.org/wiki/Q3041255",
               "schema:name": "SINTEF"
            },
            "hasPositiveElectrode": {
                "@type": "Electrode",
                "hasActiveMaterial": {
                    "@type": "LithiumIronPhosphate"
                }
            },
            "hasNegativeElectrode": {
                "@type": "Electrode",
                "hasActiveMaterial": {
                    "@type": "Graphite"
                }
            },
            "hasCase": {
                "@type": "R2032"
            },
            "hasProperty": {
               "@type": ["NominalVoltage", "ConventionalProperty"],
               "hasNumericalPart": {
                     "@type": "RealData",
                     "hasNumberValue": 3.2
               },
               "hasMeasurementUnit": "emmo:Volt"
            }
         }

jsonld_LNOGr = {
            "@context": "https://w3id.org/emmo/domain/battery/context",
            "@type": "BatteryCell",
            "schema:name": "My LNO-Graphite R2032 Coin Cell",
            "schema:manufacturer": {
               "@id": "https://www.wikidata.org/wiki/Q3041255",
               "schema:name": "SINTEF"
            },
            "hasPositiveElectrode": {
                "@type": "Electrode",
                "hasActiveMaterial": {
                    "@type": "LithiumNickelOxide"
                }
            },
            "hasNegativeElectrode": {
                "@type": "Electrode",
                "hasActiveMaterial": {
                    "@type": "Graphite"
                }
            },
            "hasCase": {
                "@type": "R2032"
            },
            "hasProperty": {
               "@type": ["NominalVoltage", "ConventionalProperty"],
               "hasNumericalPart": {
                     "@type": "RealData",
                     "hasNumberValue": 3.6
               },
               "hasMeasurementUnit": "emmo:Volt"
            }
         }

## Parse JSON-LD into an RDF Graph

We now create a new RDFLib graph and load the JSON-LD descriptions into it. This converts each JSON-LD block into RDF triples — the building blocks of the Semantic Web. You can then query this graph using SPARQL to retrieve specific metadata or relationships.

In [31]:
import json
import rdflib
import requests
import sys
from IPython.display import Image, display
import matplotlib.pyplot as plt

We create the graph using a very handy python package called [rdflib](https://rdflib.readthedocs.io/en/stable/), which provides us a way to parse our json-ld data, run some queries using the language [SPARQL](https://en.wikipedia.org/wiki/SPARQL), and serialize the graph in any RDF compatible format (e.g. JSON-LD, Turtle, etc.).

In [32]:
# Create a new graph
g = rdflib.Graph()

# Parse our json-ld data into the graph
g.parse(data=json.dumps(jsonld_LFPGr), format="json-ld")
g.parse(data=json.dumps(jsonld_LNOGr), format="json-ld")

<Graph identifier=Nedd5721389ac416c8e39b3018726d4e6 (<class 'rdflib.graph.Graph'>)>

## Query All Triples in the Graph

We begin by running a simple SPARQL query that returns all triples (`subject → predicate → object`) in the graph. This is useful for inspecting what data was parsed and verifying that your JSON-LD was interpreted correctly.


In [33]:
# Create a SPARQL query to return all the triples in the graph
query_all = """
SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object
}
"""

# Execute the SPARQL query
all_the_things = g.query(query_all)

# Print the results
for row in all_the_things:
    print(row)


(rdflib.term.BNode('Nf7409c01f9d148dbb30f671e5d555a12'), rdflib.term.URIRef('https://w3id.org/emmo#EMMO_faf79f53_749d_40b2_807c_d34244c192f4'), rdflib.term.Literal('3.6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.BNode('N99f92e57532843d69c8cae74f27272c3'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://w3id.org/emmo/domain/chemical-substance#substance_d53259a7_0d9c_48b9_a6c1_4418169df303'))
(rdflib.term.BNode('Nbac6e3c31ec9453a82b9813ef6896366'), rdflib.term.URIRef('https://w3id.org/emmo#EMMO_faf79f53_749d_40b2_807c_d34244c192f4'), rdflib.term.Literal('3.2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.BNode('Nb50ad5f394e34bef92855a353bac2ebf'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://w3id.org/emmo/domain/electrochemistry#electrochemistry_0f007072_a8dd_4798_b865_1bf9363be627'))
(rdflib.term.BN

You can see that our human-readable JSON-LD file has been transformed into some nasty looking (but machine-readable!) triples.

## Filtering Battery Cells by Voltage

You can define a SPARQL query to return only cells with a `NominalVoltage` greater than 3.5 V. This demonstrates how JSON-LD + RDF can support rich filtering and data analytics with minimal effort.

In [34]:
from ontopy import get_ontology

# Loading from web
battinfo = get_ontology('https://w3id.org/emmo/domain/battery/inferred').load()

query = f"""
PREFIX schema: <https://schema.org/>
PREFIX emmo: <http://emmo.info/emmo#>

SELECT ?cellName WHERE {{
    ?cell a <{battinfo.BatteryCell.iri}>;
          schema:name ?cellName;
          <{battinfo.hasProperty.iri}> ?property.

    ?property a <{battinfo.NominalVoltage.iri}>;
              <{battinfo.hasNumericalPart.iri}> ?numericalPart.

    ?numericalPart <{battinfo.hasNumberValue.iri}> ?voltage.

    FILTER (?voltage > 3.5)
}}
"""

# Execute the SPARQL query
results = g.query(query)

# Print the results
for row in results:
    print(row)


(rdflib.term.Literal('My LNO-Graphite R2032 Coin Cell'),)


## Fetch additional information from other sources [Advanced]

Ontologies contain a lot of information about the meaning of things, but they don't always contain an exhaustive list of all the properties. Instead, they often point to other sources where that information exists rather than duplicating it. Let's see how you can use the ontology to fetch additional information from other sources.

In [35]:
# Query the ontology to find the wikidata id for LithiumNickelOxide
query = f"""
SELECT ?wikidataId
WHERE {{
    <{battinfo.LithiumNickelOxide.iri}> <{battinfo.wikidataReference.iri}> ?wikidataId .
}}
"""

qres = g.query(query)
for row in qres:
    wikidata_id = row.wikidataId.split('/')[-1]

print(f"The PubChem ID of Lithiun Nickel Oxide is: {wikidata_id}")

The PubChem ID of Lithiun Nickel Oxide is: Q81988484


Finally, let's retireve more information about Lithium Nickel Oxide from Wikidata and PubChem

In [36]:
# Query the Wikidata knowledge graph for more information
wikidata_endpoint = "https://query.wikidata.org/sparql"

# SPARQL query to get the PubChem ID
query = """
SELECT ?id WHERE {
  wd:%s wdt:P662 ?id .
}
""" % wikidata_id

# Execute the request
response = requests.get(wikidata_endpoint, params={'query': query, 'format': 'json'})
data = response.json()

# Extract and display the PubChem ID
if data['results']['bindings']:
    PubChemId = data['results']['bindings'][0]['id']['value']
    print(f"The PubChem ID for a LithiumNickelOxide cell: {PubChemId}")

else:
    print("None found.")

The PubChem ID for a LithiumNickelOxide cell: 138395181


In [37]:
def get_pubchem_compound_data(cid):
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    compound_url = f"{base_url}/compound/cid/{cid}/JSON"
    response = requests.get(compound_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Fetch data for the compound with CID 138395181
compound_data = get_pubchem_compound_data(PubChemId)
if compound_data:
    pretty_json = json.dumps(compound_data, indent=4)  # Pretty-print the JSON data
    print(pretty_json)
else:
    print("Data not found or error in API request.")

{
    "PC_Compounds": [
        {
            "id": {
                "id": {
                    "cid": 138395181
                }
            },
            "atoms": {
                "aid": [
                    1,
                    2,
                    3,
                    4
                ],
                "element": [
                    28,
                    8,
                    8,
                    3
                ],
                "charge": [
                    {
                        "aid": 1,
                        "value": 2
                    },
                    {
                        "aid": 2,
                        "value": -2
                    },
                    {
                        "aid": 3,
                        "value": -2
                    },
                    {
                        "aid": 4,
                        "value": 1
                    }
                ]
            },
            "coords": [
            

## Summary and Next Steps

In this notebook, we demonstrated how to:
- Create semantically rich battery cell descriptions using JSON-LD
- Use ontology terms from EMMO and Schema.org
- Convert the data to RDF using RDFLib
- Run SPARQL queries to inspect and analyze structured data

This approach is ideal for:
- Sharing metadata in a standardized way
- Integrating battery data into digital twins or modeling platforms
- Building searchable knowledge graphs for research

### Next Steps:
- Add more properties like capacity, electrolyte, or separator
- Validate JSON-LD structure with JSON Schema or SHACL
- Extend to test protocols and datasets